In [1]:
#All_samples_extracted_3UTR_coverage_dict=All_samples_Target_3UTR_coverages
# np.array(All_Samples_Total_depth),
# UTR_events_dict

In [2]:
import numpy as np
import os
import sys
import datetime

import scipy as sp
import scipy.stats

from bisect import bisect

import math

In [3]:
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Group1_Tophat_aligned_file=''
    Group2_Tophat_aligned_file=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Num_least_in_group1_local=''
    Num_least_in_group2_local=''
    Coverage_cutoff_local = ''
    FDR_cutoff_local = ''
    Fold_change_cutoff_local = ''
    PDUI_cutoff_local = ''
    
    for line in open(cfg_file,'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip();
            command = line.split('=');
            if command[0] == 'Group1_Tophat_aligned_Wig':
                Group1_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Group2_Tophat_aligned_Wig':
                Group2_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            
            ##Parameters
            if command[0] == 'Num_least_in_group1':
                Num_least_in_group1_local = command[1]
            if command[0] == 'Num_least_in_group2':
                Num_least_in_group2_local = command[1]
            if command[0] == 'Coverage_cutoff':
                Coverage_cutoff_local = command[1]
            if command[0] == 'FDR_cutoff':
                FDR_cutoff_local = command[1]
            if command[0] == 'Fold_change_cutoff':
                Fold_change_cutoff_local = command[1]
            if command[0] == 'PDUI_cutoff':
                PDUI_cutoff_local = command[1]
            
    
    if Group1_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 1!", file=sys.stderr)
        exit(1)
    if Group2_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 2!", file=sys.stderr)
        exit(1)
    if output_directory=='':
        print("No output directory!", file=sys.stderr)
        exit(1)
    if Annotated_3UTR_file=='':
        print("No annotated 3' UTR file!", file=sys.stderr)
        exit(1)
    if Output_result_file=='':
        print("No result file name!", file=sys.stderr)
        exit(1)
    return Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local


In [4]:
(Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,
 output_directory,
 Annotated_3UTR_file,
 Output_result_file,
 Num_least_in_group1_local,Num_least_in_group2_local,
 Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,
 PDUI_cutoff_local) = parse_cfgfile('/home/li/桌面/PROJECT6/apasite_predict2/DAPARS/DATA/DaPars_test_data_configure.txt')

In [5]:
Group1_Tophat_aligned_file=['../DATA/Condition_A_chrX.wig']
num_group_1 = len(Group1_Tophat_aligned_file)

In [6]:
Annotated_3UTR_file='../DATA/hg19_refseq_extracted_3UTR.bed'

In [7]:
# All_Sample_files = Group1_Tophat_aligned_file[:]
# All_Sample_files.extend(Group2_Tophat_aligned_file)
All_Sample_files=['../DATA/Condition_A_chrX.wig', '../DATA/Condition_B_chrX.wig']

In [8]:
#Annotated_3UTR_file

for line in open(Annotated_3UTR_file,'r'):
    # print(line)
    #strip() 方法用于移除字符串头尾指定的字符
    fields = line.strip('\n').split('\t')
    print(fields)
    break

['chr14', '50792327', '50792946', 'NM_001003805|ATP5S|chr14|+', '0', '+']


In [ ]:
UTR_events_dict = {}

for line in open(Annotated_3UTR_file,'r'):
    fields = line.strip('\n').split('\t')
    curr_chr = fields[0]
    region_start = int(float(fields[1]))
    region_end   = int(float(fields[2]))
    curr_strand  = fields[-1]

    UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
    end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
    if curr_strand == '+':
        region_end = str(int(region_end) - end_shift)
    else:
        region_start = str(int(region_start) + end_shift)
    region_start = int(region_start) + 1
    region_end   = int(region_end) - 1
    if region_start + 50 < region_end:
        UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]


In [54]:
len(UTR_events_dict)

24983

In [55]:
for KK in UTR_events_dict:
    print(KK)
    break

NM_001003805|ATP5S|chr14|+


In [56]:
for KK in open(Annotated_3UTR_file,'r'):
    print(KK)
    break

chr14	50792327	50792946	NM_001003805|ATP5S|chr14|+	0	+



In [58]:
All_Samples_Total_depth = []

for line in open(curr_wig_file,'r'):

    if '#' not in line and line[0:3] == 'chr':
        fields = line.strip('\n').split('\t')
        chrom_name = fields[0]
        region_start = int(float(fields[1]))
        region_end = int(float(fields[2]))
        cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
        if chrom_name not in curr_sample_All_chroms_coverage_dict:
            curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]

        if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
            curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
            curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
        curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
        curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))
    num_line += 1
curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
All_Samples_Total_depth.append(cur_sample_total_depth)

In [53]:
curr_chr = fields[0]
region_start = int(float(fields[1]))
region_end   = int(float(fields[2]))
UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
UTR_pos

'chr7:105096960-105098373'

In [59]:
for curr_3UTR_event_id in UTR_events_dict:
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
    curr_chr = curr_3UTR_structure[0]

    if curr_chr in curr_sample_All_chroms_coverage_dict:
        curr_chr_coverage = curr_sample_All_chroms_coverage_dict[curr_chr]
        region_start = curr_3UTR_structure[1]
        region_end = curr_3UTR_structure[2]
        left_region_index = bisect(curr_chr_coverage[0],region_start)
        right_region_index = bisect(curr_chr_coverage[0],region_end)

        extracted_coverage = curr_chr_coverage[1][left_region_index:right_region_index+1]
        extracted_3UTR_region = curr_chr_coverage[0][left_region_index:right_region_index]
        extracted_3UTR_region.insert(0,region_start)
        extracted_3UTR_region.append(region_end)
        
        if curr_3UTR_event_id not in All_samples_extracted_3UTR_coverage_dict:
            All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id] = []
        All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id].append([extracted_coverage,extracted_3UTR_region])


In [62]:
Annotated_3UTR_file

'../DATA/hg19_refseq_extracted_3UTR.bed'

In [60]:
for KK in All_samples_extracted_3UTR_coverage_dict:
    print(KK)
    break

NM_001468|GAGE1|chrX|+


In [10]:
end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
end_shift

124

In [11]:
## 加载所有样品的覆盖率
## Load coverage for all samples
All_samples_extracted_3UTR_coverage_dict = {}
for curr_wig_file in All_Sample_files:
    break
curr_wig_file

'../DATA/Condition_A_chrX.wig'

In [50]:
for line in open(curr_wig_file,'r'):
    print(line)
    break

chrX	0	193010	0



In [ ]:
fields = line.strip('\n').split('\t')
chrom_name = fields[0]


In [12]:
UTR_events_dict = {}

In [13]:
for line in open(Annotated_3UTR_file,'r'):
    break
line

'chr14\t50792327\t50792946\tNM_001003805|ATP5S|chr14|+\t0\t+\n'

In [14]:
fields = line.strip('\n').split('\t')
fields

['chr14', '50792327', '50792946', 'NM_001003805|ATP5S|chr14|+', '0', '+']

In [15]:
chrom_name = fields[0]
chrom_name

'chr14'

In [16]:
region_start = int(float(fields[1]))
region_end   = int(float(fields[2]))
region_end

50792946

In [17]:
curr_strand  = fields[-1]
curr_strand

'+'

In [18]:
fields[3]

'NM_001003805|ATP5S|chr14|+'

In [19]:
UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
UTR_pos

'chr14:50792327-50792946'

In [20]:
end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))

In [21]:
if curr_strand == '+':
    region_end = str(int(region_end) - end_shift)
else:
    region_start = str(int(region_start) + end_shift)
region_start = int(region_start) + 1
region_end   = int(region_end) - 1
if region_start + 50 < region_end:
    UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]


In [22]:
UTR_events_dict[fields[3]]

['chr14', 50792328, 50792821, '+', 'chr14:50792327-50792946']

In [23]:
fields[3]

'NM_001003805|ATP5S|chr14|+'

In [24]:
UTR_events_dict

{'NM_001003805|ATP5S|chr14|+': ['chr14',
  50792328,
  50792821,
  '+',
  'chr14:50792327-50792946']}

In [25]:
type(UTR_events_dict)

dict

In [26]:
All_samples_extracted_3UTR_coverage_dict = {}
curr_sample_All_chroms_coverage_dict = {}

In [27]:
for curr_wig_file in All_Sample_files:
    break
curr_wig_file

'../DATA/Condition_A_chrX.wig'

In [28]:
for line in open(curr_wig_file,'r'):
    break
line

'chrX\t0\t193010\t0\n'

In [29]:
num_line = 0
cur_sample_total_depth = 0


if '#' not in line and line[0:3] == 'chr':
    fields = line.strip('\n').split('\t')
    chrom_name = fields[0]
    region_start = int(float(fields[1]))
    region_end = int(float(fields[2]))
    cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
    if chrom_name not in curr_sample_All_chroms_coverage_dict:
        curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]
        
    if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
        curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
        curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
    curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
    curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))

In [30]:
chrom_name[0][-1]

'c'

In [31]:
curr_sample_All_chroms_coverage_dict

{'chrX': [[0, 193010], [0, 0]]}

In [32]:
UTR_events_dict

{'NM_001003805|ATP5S|chr14|+': ['chr14',
  50792328,
  50792821,
  '+',
  'chr14:50792327-50792946']}

In [33]:
for line in open(Annotated_3UTR_file,'r'):
    fields = line.strip('\n').split('\t')
    curr_chr = fields[0]
    region_start = int(float(fields[1]))
    region_end   = int(float(fields[2]))
    curr_strand  = fields[-1]

    UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
    end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
    if curr_strand == '+':
        region_end = str(int(region_end) - end_shift)
    else:
        region_start = str(int(region_start) + end_shift)
    region_start = int(region_start) + 1
    region_end   = int(region_end) - 1
    
    # 全部满足
    if region_start + 50 < region_end:
        # print('if region_start + 50 < region_end:')
        UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]



In [34]:
#全
len(UTR_events_dict)

24983

In [35]:
#当前样本
#当前样本
curr_wig_file

'../DATA/Condition_A_chrX.wig'

In [36]:
All_samples_extracted_3UTR_coverage_dict = {}


curr_sample_All_chroms_coverage_dict = {}
num_line = 0
cur_sample_total_depth = 0
for line in open(curr_wig_file,'r'):

    if '#' not in line and line[0:3] == 'chr':
        fields = line.strip('\n').split('\t')
        chrom_name = fields[0]
        region_start = int(float(fields[1]))
        region_end = int(float(fields[2]))
        cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
        if chrom_name not in curr_sample_All_chroms_coverage_dict:
            curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]

        if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
            curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
            curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
        curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
        curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))
    num_line += 1
curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
# All_Samples_Total_depth.append(cur_sample_total_depth)

In [37]:
len(curr_sample_All_chroms_coverage_dict['chrX'])

2

In [38]:
for KK in curr_sample_All_chroms_coverage_dict:
    print(KK)

chrX


In [39]:
for curr_3UTR_event_id in UTR_events_dict:
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
    curr_chr = curr_3UTR_structure[0]
    break
    

curr_3UTR_structure

['chr14', 50792328, 50792821, '+', 'chr14:50792327-50792946']

In [40]:
for curr_3UTR_event_id in UTR_events_dict:
    print(curr_3UTR_event_id)
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
    curr_chr = curr_3UTR_structure[0]
    print(curr_chr)
    if curr_chr=='chrX':
        break
    

NM_001003805|ATP5S|chr14|+
chr14
NM_001003800|BICD2|chr9|-
chr9
NM_001008781|FAT3|chr11|+
chr11
NM_001008783|SLC35D3|chr6|+
chr6
NR_003227|AFG3L1P|chr16|+
chr16
NR_003226|AFG3L1P|chr16|+
chr16
NR_003225|LGALS3|chr14|+
chr14
NR_003224|SNORD114-31|chr14|+
chr14
NR_003223|SNORD114-30|chr14|+
chr14
NR_003222|SNORD114-29|chr14|+
chr14
NR_003221|SNORD114-28|chr14|+
chr14
NR_003220|SNORD114-27|chr14|+
chr14
NR_003229|SNORD113-1|chr14|+
chr14
NR_003228|AFG3L1P|chr16|+
chr16
NM_001468|GAGE1|chrX|+
chrX


In [41]:
if curr_chr in curr_sample_All_chroms_coverage_dict:
    curr_chr_coverage = curr_sample_All_chroms_coverage_dict[curr_chr]
    region_start = curr_3UTR_structure[1]
    region_end = curr_3UTR_structure[2]


In [42]:
len(curr_chr_coverage)

2

In [43]:
curr_chr_coverage[0]

[0,
 193010,
 193020,
 193031,
 193032,
 193034,
 193037,
 193039,
 193044,
 193047,
 193050,
 193051,
 193053,
 193054,
 193056,
 193057,
 193058,
 193061,
 193662,
 193722,
 197848,
 197860,
 197865,
 197868,
 197876,
 197881,
 197883,
 197884,
 197885,
 197887,
 197888,
 197894,
 197897,
 197907,
 197912,
 197918,
 197919,
 197923,
 197928,
 197953,
 197961,
 197977,
 197984,
 197985,
 198017,
 198019,
 198020,
 198022,
 198024,
 198029,
 198043,
 198045,
 198050,
 198054,
 198056,
 198057,
 198061,
 198062,
 198063,
 198065,
 198067,
 198068,
 198069,
 198070,
 198072,
 198074,
 198078,
 198080,
 198082,
 198083,
 198087,
 198088,
 198090,
 198092,
 198093,
 198095,
 198098,
 198099,
 198101,
 198102,
 198104,
 198105,
 198106,
 198107,
 198108,
 198109,
 198110,
 198111,
 198112,
 198113,
 198114,
 198116,
 198118,
 198120,
 198122,
 198123,
 198125,
 198129,
 198130,
 198133,
 198134,
 198135,
 198136,
 198137,
 198140,
 198141,
 198144,
 198145,
 198146,
 198147,
 198148,
 19814

In [44]:
curr_chr_coverage[1]

[0,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 21,
 23,
 25,
 26,
 0,
 1,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 7,
 8,
 9,
 8,
 7,
 6,
 5,
 6,
 5,
 4,
 5,
 4,
 3,
 4,
 5,
 6,
 5,
 6,
 7,
 20,
 23,
 24,
 25,
 28,
 37,
 38,
 39,
 40,
 41,
 43,
 45,
 47,
 48,
 58,
 59,
 61,
 67,
 68,
 69,
 70,
 71,
 72,
 76,
 78,
 79,
 80,
 84,
 85,
 86,
 88,
 92,
 100,
 104,
 107,
 110,
 114,
 115,
 117,
 120,
 121,
 123,
 124,
 126,
 127,
 131,
 132,
 133,
 134,
 135,
 134,
 135,
 134,
 139,
 171,
 173,
 174,
 173,
 174,
 179,
 180,
 188,
 191,
 192,
 194,
 199,
 187,
 185,
 189,
 190,
 191,
 196,
 195,
 186,
 188,
 192,
 194,
 201,
 204,
 207,
 209,
 210,
 209,
 216,
 224,
 226,
 224,
 222,
 223,
 226,
 228,
 222,
 220,
 221,
 228,
 236,
 237,
 236,
 234,
 236,
 243,
 242,
 244,
 240,
 238,
 240,
 238,
 234,
 227,
 222,
 219,
 221,
 220,
 221,
 216,
 218,
 217,
 216,
 215,
 228,
 230,
 233,
 239,
 245,
 243,
 245,
 246,
 244,
 240,
 243,
 236,


In [45]:
region_start

49370597

In [46]:
curr_chr_coverage

[[0,
  193010,
  193020,
  193031,
  193032,
  193034,
  193037,
  193039,
  193044,
  193047,
  193050,
  193051,
  193053,
  193054,
  193056,
  193057,
  193058,
  193061,
  193662,
  193722,
  197848,
  197860,
  197865,
  197868,
  197876,
  197881,
  197883,
  197884,
  197885,
  197887,
  197888,
  197894,
  197897,
  197907,
  197912,
  197918,
  197919,
  197923,
  197928,
  197953,
  197961,
  197977,
  197984,
  197985,
  198017,
  198019,
  198020,
  198022,
  198024,
  198029,
  198043,
  198045,
  198050,
  198054,
  198056,
  198057,
  198061,
  198062,
  198063,
  198065,
  198067,
  198068,
  198069,
  198070,
  198072,
  198074,
  198078,
  198080,
  198082,
  198083,
  198087,
  198088,
  198090,
  198092,
  198093,
  198095,
  198098,
  198099,
  198101,
  198102,
  198104,
  198105,
  198106,
  198107,
  198108,
  198109,
  198110,
  198111,
  198112,
  198113,
  198114,
  198116,
  198118,
  198120,
  198122,
  198123,
  198125,
  198129,
  198130,
  198133,
  198

In [47]:
left_region_index = bisect(curr_chr_coverage[0],region_start)
left_region_index

550250